In [10]:
#!/usr/bin/env python3
"""
Algoritmo genético para encontrar una palabra objetivo.
- Genera una población aleatoria de 50 palabras (por defecto).
- Reproduce por parlante (TTS) la población inicial y luego la mejor palabra de cada generación.
- Función de aptitud: similitud con la palabra objetivo (por defecto: "algoritmo").

Requisitos:
    pip install pyttsx3

Uso:
    python algoritmo_genetico_palabras.py --target algoritmo --pop-size 50

Hecho en español para que sea fácil de entender.
"""

import random
import string
import time
import argparse
import sys

# Intentamos usar pyttsx3 para TTS (offline). Si no está instalado, el script imprimirá en consola en su lugar.
try:
    import pyttsx3
    TTS_AVAILABLE = True
except Exception:
    TTS_AVAILABLE = False


ALPHABET = string.ascii_lowercase + ' '  # letras permitidas (minusculas) y espacio


def random_word(length):
    return ''.join(random.choice(ALPHABET) for _ in range(length))


def levenshtein(a: str, b: str) -> int:
    """Distancia de Levenshtein (programación dinámica)."""
    if a == b:
        return 0
    la, lb = len(a), len(b)
    if la == 0:
        return lb
    if lb == 0:
        return la
    prev = list(range(lb + 1))
    for i, ca in enumerate(a, start=1):
        cur = [i] + [0] * lb
        for j, cb in enumerate(b, start=1):
            add = prev[j] + 1
            delete = cur[j - 1] + 1
            sub = prev[j - 1] + (0 if ca == cb else 1)
            cur[j] = min(add, delete, sub)
        prev = cur
    return prev[lb]


def fitness(word: str, target: str) -> float:
    """Devuelve fitness en [0,1]. 1.0 = palabra exacta."""
    if len(word) == len(target):
        matches = sum(1 for a, b in zip(word, target) if a == b)
        return matches / max(1, len(target))
    else:
        d = levenshtein(word, target)
        maxlen = max(len(word), len(target))
        return max(0.0, 1.0 - d / maxlen)


def tournament_selection(population, scores, k=5):
    chosen_ix = random.sample(range(len(population)), k)
    best = max(chosen_ix, key=lambda i: scores[i])
    return population[best]


def one_point_crossover(a: str, b: str) -> str:
    if len(a) != len(b):
        b = (b * ((len(a) // len(b)) + 1))[: len(a)]
    point = random.randint(1, len(a) - 1)
    return a[:point] + b[point:]


def mutate(word: str, mutation_rate: float) -> str:
    out = list(word)
    for i in range(len(out)):
        if random.random() < mutation_rate:
            out[i] = random.choice(ALPHABET)
    return ''.join(out)


class Speaker:
    def __init__(self, rate=160):
        self.available = TTS_AVAILABLE
        if self.available:
            try:
                self.engine = pyttsx3.init()
                self.engine.setProperty('rate', rate)
            except Exception:
                self.available = False
                self.engine = None
        else:
            self.engine = None

    def say(self, text: str):
        if self.available and self.engine:
            try:
                self.engine.say(text)
                self.engine.runAndWait()
            except Exception:
                print(f"[TTS falló] {text}")
        else:
            print(f"[TTS no disponible] {text}")


def run_genetic(target: str,
                pop_size=50,
                mutation_rate=0.02,
                max_gens=1000,
                elitism=1,
                tournament_k=5,
                speak_initial=True,
                speak_best_each_gen=True,
                seed=None):
    if seed is not None:
        random.seed(seed)

    target = target.lower()
    gene_length = len(target)

    population = [random_word(gene_length) for _ in range(pop_size)]

    speaker = Speaker()

    if speak_initial:
        print("Reproduciendo población inicial...")
        for w in population:
            speaker.say(w)
            time.sleep(0.02)

    scores = [fitness(w, target) for w in population]

    best_word = population[max(range(len(population)), key=lambda i: scores[i])]
    best_score = max(scores)

    print(f"Generación 0: mejor='{best_word}' fitness={best_score:.4f}")

    if best_score >= 1.0:
        print("¡La palabra objetivo ya estaba en la población inicial!")
        return best_word, 0

    for gen in range(1, max_gens + 1):
        new_pop = []

        ranked = sorted(zip(population, scores), key=lambda x: x[1], reverse=True)
        elites = [w for w, s in ranked[:elitism]]
        new_pop.extend(elites)

        while len(new_pop) < pop_size:
            parent1 = tournament_selection(population, scores, k=tournament_k)
            parent2 = tournament_selection(population, scores, k=tournament_k)
            child = one_point_crossover(parent1, parent2)
            child = mutate(child, mutation_rate)
            new_pop.append(child)

        population = new_pop
        scores = [fitness(w, target) for w in population]

        best_idx = max(range(len(population)), key=lambda i: scores[i])
        best_word = population[best_idx]
        best_score = scores[best_idx]
        avg_score = sum(scores) / len(scores)

        print(f"Generación {gen}: mejor='{best_word}' fitness={best_score:.4f}  promedio={avg_score:.4f}")

        if speak_best_each_gen:
            speaker.say(best_word)

        if best_score >= 1.0:
            print(f"¡Objetivo encontrado en la generación {gen}: '{best_word}'!")
            return best_word, gen

    print("Se alcanzó el máximo de generaciones sin encontrar la palabra exacta.")
    return best_word, max_gens


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Algoritmo genético para encontrar una palabra objetivo y reproducirla por parlante.')
    parser.add_argument('--target', type=str, default='algoritmo', help='Palabra objetivo (por defecto: "algoritmo")')
    parser.add_argument('--pop-size', type=int, default=50, help='Tamaño de la población')
    parser.add_argument('--mut-rate', type=float, default=0.02, help='Tasa de mutación por carácter (ej: 0.02)')
    parser.add_argument('--max-gens', type=int, default=1000, help='Máximo de generaciones')
    parser.add_argument('--no-speak-initial', action='store_true', help='No reproducir la población inicial')
    parser.add_argument('--no-speak-best', action='store_true', help='No reproducir la mejor palabra de cada generación')
    parser.add_argument('--seed', type=int, default=None, help='Semilla aleatoria (opcional)')

    # 🔧 Fix para Jupyter: limpiar sys.argv para que argparse no falle con "--f"
    sys.argv = ["algoritmo_genetico_palabras.py"]

    args = parser.parse_args()

    run_genetic(target=args.target,
                pop_size=args.pop_size,
                mutation_rate=args.mut_rate,
                max_gens=args.max_gens,
                speak_initial=not args.no_speak_initial,
                speak_best_each_gen=not args.no_speak_best,
                seed=args.seed)


Reproduciendo población inicial...
[TTS no disponible]  ltzvbnfr
[TTS no disponible] sfjrytcms
[TTS no disponible] k cp hgkh
[TTS no disponible] kymbclrtf
[TTS no disponible] rd ibzvnr
[TTS no disponible] vdsk dyqo
[TTS no disponible] bwhtprara
[TTS no disponible] fahuvqpxo
[TTS no disponible]  ryldaywq
[TTS no disponible] v xth sog
[TTS no disponible] kunrpi ma
[TTS no disponible] nxxqrapei
[TTS no disponible] ablfeoftq
[TTS no disponible] jcxx jvrt
[TTS no disponible]  xoperlfh
[TTS no disponible] bcqfekkkl
[TTS no disponible] natawpoov
[TTS no disponible]  ao ibbdi
[TTS no disponible] hpbypvlmk
[TTS no disponible] xucxuvvvj
[TTS no disponible] jcumngmzo
[TTS no disponible] hhewfodfp
[TTS no disponible] oxgcbzpvi
[TTS no disponible] hsexphxcb
[TTS no disponible] hmgkdxjcn
[TTS no disponible] nfaojumia
[TTS no disponible] wwbgbsjvd
[TTS no disponible] mrlcemydg
[TTS no disponible] qrmbqxqis
[TTS no disponible] frtwvaxgi
[TTS no disponible] jksuywvyd
[TTS no disponible] ildemohqt
[TTS 